<a href="https://www.kaggle.com/code/yorkyong/m5-forecasting-autogluon-1series?scriptVersionId=205347718" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div class ="alert alert-block alert-warning">
    
- This notebook aims to apply AutoGluon-TimeSeries as covered by https://paperswithcode.com/paper/autogluon-timeseries-automl-for-probabilistic on the M5 dataset

- Research Question: How can an automated machine learning framework enhance accuracy and efficiency of demand forecasting?
    
- We attempt to implement this on the private and public leaderboard and compare the accuracy accordingly.

- Speed of inference and training will be noted using `%time` for recording purpose


# Import Libraries

In [4]:
pip install autogluon thinc==8.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of spacy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to de

In [1]:
import gc
import psutil

import pandas as pd  # For data manipulation
import numpy as np  # For numerical operations
import warnings  # To suppress warnings
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.timeseries.utils.forecast import get_forecast_horizon_index_ts_dataframe
from sklearn.preprocessing import LabelEncoder
import re

warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

import random  # For generating random numbers

# Function to set a fixed random seed for reproducibility
def seed_everything(seed):
    np.random.seed(seed)  # Set numpy random seed
    random.seed(seed)  # Set built-in random seed

seed_everything(seed=2024)  # Set the seed to 2024


# Load Dataset

In [42]:
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")  # Load calendar dataset
print(f"len(calendar):{len(calendar)}")  # Print the number of rows in calendar
calendar

len(calendar):1969


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [43]:
sales_train_evaluation = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
print(f"len(sales_train_evaluation): {len(sales_train_evaluation)}")
sales_train_evaluation.head()

len(sales_train_evaluation): 30490


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [44]:
sell_prices = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
print(f"len(sell_prices):{len(sell_prices)}")
sell_prices

len(sell_prices):6841121


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [3]:
# Enhanced memory optimization function with object datatype handling
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2  # Initial memory usage in MB
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:  # Downcast numerics
            c_min, c_max = df[col].min(), df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
        elif col_type == 'object':  # Handle object types
            if col == 'date':  # Convert date column to datetime
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            else:
                df[col] = df[col].astype('category')  # Convert other object types to category
    end_mem = df.memory_usage().sum() / 1024**2  # Final memory usage in MB
    if verbose:
        print(f'Memory usage reduced to {end_mem:5.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df


In [5]:
# Apply the optimized memory reduction function to each dataframe
calendar = reduce_mem_usage(calendar)
sell_prices = reduce_mem_usage(sell_prices)
sales_train_evaluation = reduce_mem_usage(sales_train_evaluation)

# Testing AutoGluon on one specific item

In [47]:
# Specify the item_id, dept_id, cat_id, store_id you want to filter
item_id = 'FOODS_3_586'
dept_id = 'FOODS_3'
cat_id = 'FOODS'
store_id = 'TX_1'

In [48]:
# Select a specific series (adjust item_id, dept_id, cat_id, and store_id as required)
selected_item = sales_train_evaluation[(sales_train_evaluation['item_id'] == item_id) &
                                       (sales_train_evaluation['dept_id'] == dept_id) &
                                       (sales_train_evaluation['cat_id'] == cat_id) &
                                       (sales_train_evaluation['store_id'] == store_id)]


In [49]:
selected_item

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
15006,FOODS_3_586_TX_1_evaluation,FOODS_3_586,FOODS_3,FOODS,TX_1,TX,53,46,42,45,...,61,70,73,46,56,49,46,64,70,69


In [56]:
# Map d_ columns to dates
date_mapping = calendar[['date', 'd']].set_index('d').to_dict()['date']
sales_data = selected_item.filter(regex=r'd_\d+', axis=1).T
sales_data.columns = ['sales']
sales_data['timestamp'] = sales_data.index.map(date_mapping)
sales_data['timestamp'] = pd.to_datetime(sales_data['timestamp'])
sales_data['item_id'] = selected_item['id'].iloc[0]
sales_data = TimeSeriesDataFrame(sales_data.set_index(['item_id', 'timestamp']))

In [57]:
sales_data

sales
item_id                     timestamp        
FOODS_3_586_TX_1_evaluation 2011-01-29     53
                            2011-01-30     46
                            2011-01-31     42
                            2011-02-01     45
                            2011-02-02     28
...                                       ...
                            2016-05-18     49
                            2016-05-19     46
                            2016-05-20     64
                            2016-05-21     70
                            2016-05-22     69

[1941 rows x 1 columns]

# Setting up Time varying covariates

In [58]:
# Filter sell_prices for the relevant store_id and item_id
filtered_sell_prices = sell_prices[(sell_prices['store_id'] == store_id) & (sell_prices['item_id'] == item_id)]

# Encode event-related features in the calendar dataframe
event_columns = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
le = LabelEncoder()
for col in event_columns:
    calendar[col] = le.fit_transform(calendar[col].astype(str))

# Merge calendar with filtered_sell_prices on wm_yr_wk
calendar = calendar.merge(filtered_sell_prices, how='left', on='wm_yr_wk')

In [59]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,store_id,item_id,sell_price
0,2011-01-29,11101,Saturday,1,1,2011,d_1,30,4,4,2,0,0,0,TX_1,FOODS_3_586,1.480469
1,2011-01-30,11101,Sunday,2,1,2011,d_2,30,4,4,2,0,0,0,TX_1,FOODS_3_586,1.480469
2,2011-01-31,11101,Monday,3,1,2011,d_3,30,4,4,2,0,0,0,TX_1,FOODS_3_586,1.480469
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,30,4,4,2,1,1,0,TX_1,FOODS_3_586,1.480469
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,30,4,4,2,1,0,1,TX_1,FOODS_3_586,1.480469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,30,4,4,2,0,1,1,TX_1,FOODS_3_586,1.679688
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,30,4,4,2,0,0,0,TX_1,FOODS_3_586,1.679688
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,30,4,4,2,0,0,0,TX_1,FOODS_3_586,1.679688
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,30,4,4,2,0,0,0,TX_1,FOODS_3_586,1.679688


In [60]:
# Reset the index for sales_data to ensure it has a 'timestamp' column for merging
sales_data_reset = sales_data.reset_index()

# Ensure 'date' in calendar is in datetime format and set as index
calendar['date'] = pd.to_datetime(calendar['date'])
calendar = calendar.set_index('date')

# Select only relevant columns in calendar for merging
calendar_filtered = calendar[['wday','snap_CA', 'snap_TX', 'snap_WI', 'sell_price'] + event_columns]

# Merge sales_data with calendar_filtered on timestamp
sales_data_merged = sales_data_reset.merge(calendar_filtered, left_on='timestamp', right_index=True, how='left')

# Set the index back to the original format
sales_data = sales_data_merged.set_index(['item_id', 'timestamp'])

In [61]:
sales_data

sales  wday  snap_CA  snap_TX  \
item_id                     timestamp                                   
FOODS_3_586_TX_1_evaluation 2011-01-29     53     1        0        0   
                            2011-01-30     46     2        0        0   
                            2011-01-31     42     3        0        0   
                            2011-02-01     45     4        1        1   
                            2011-02-02     28     5        1        0   
...                                       ...   ...      ...      ...   
                            2016-05-18     49     5        0        0   
                            2016-05-19     46     6        0        0   
                            2016-05-20     64     7        0        0   
                            2016-05-21     70     1        0        0   
                            2016-05-22     69     2        0        0   

                                        snap_WI  sell_price  event_name_1  \
item_id                     timestamp                                       
FOODS_3_586_TX_1_evaluation 2011-01-29        0    1.480469            30   
                            2011-01-30        0    1.480469            30   
                            2011-01-31        0    1.480469            30   
                            2011-02-01        0    1.480469            30   
                            2011-02-02        1    1.480469            30   
...                                         ...         ...           ...   
                            2016-05-18        0    1.679688            30   
                            2016-05-19        0    1.679688            30   
                            2016-05-20        0    1.679688            30   
                            2016-05-21        0    1.679688            30   
                            2016-05-22        0    1.679688            30   

                                        event_type_1  event_name_2  \
item_id                     timestamp                                
FOODS_3_586_TX_1_evaluation 2011-01-29             4             4   
                            2011-01-30             4             4   
                            2011-01-31             4             4   
                            2011-02-01             4             4   
                            2011-02-02             4             4   
...                                              ...           ...   
                            2016-05-18             4             4   
                            2016-05-19             4             4   
                            2016-05-20             4             4   
                            2016-05-21             4             4   
                            2016-05-22             4             4   

                                        event_type_2  
item_id                     timestamp                 
FOODS_3_586_TX_1_evaluation 2011-01-29             2  
                            2011-01-30             2  
                            2011-01-31             2  
                            2011-02-01             2  
                            2011-02-02             2  
...                                              ...  
                            2016-05-18             2  
                            2016-05-19             2  
                            2016-05-20             2  
                            2016-05-21             2  
                            2016-05-22             2  

[1941 rows x 10 columns]

# Initialise TimeSeriesPredictor

In [74]:
# Step 1: Initialize the TimeSeriesPredictor with a primary eval_metric
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="sales",
    known_covariates_names=['wday', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'] + event_columns,
    eval_metric="RMSSE"  # Primary metric for training
)

# Step 2: Fit the predictor
predictor.fit(train_data=sales_data, presets="best_quality", time_limit=600)

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'AutogluonModels/ag-20241105_042536'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
GPU Count:          0
Memory Avail:       26.55 GB / 31.36 GB (84.7%)
Disk Space Avail:   19.49 GB / 19.52 GB (99.9%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSSE,
 'hyperparameters': 'default',
 'known_covariates_names': ['wday',
                            'snap_CA',
                            'snap_TX',
                            'snap_WI',
                            'sell_price',
                            'event_name_1',
                            'event_type_1',
                            'event_name_2',
                            'event_type_2'],
 'num_val_windows'

In [80]:
# Step 3: Evaluate the model on additional metrics
# Generate a leaderboard with multiple evaluation metrics
leaderboard = predictor.leaderboard(
    data=sales_data,  # Use the sales data for evaluation
    display=True  # Display the leaderboard DataFrame directly
)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


                   model  score_test  score_val  pred_time_test  pred_time_val  fit_time_marginal  fit_order
0       WeightedEnsemble   -0.535829  -0.605556        3.447913       0.648620           1.262752          9
1          DirectTabular   -0.554516  -0.646693        0.141741       0.127481           3.570811          3
2             CrostonSBA   -0.582455  -0.664246        2.809778       0.024497           0.079935          4
3                   NPTS   -0.584727  -0.644715        0.636007       0.624123           0.687188          5
4                AutoETS   -0.593166  -0.655525       31.542781       0.682728           0.617166          7
5  DynamicOptimizedTheta   -0.594249  -0.656060       44.064677       1.990056           1.715187          6
6              AutoARIMA   -0.616070  -0.680747       25.580931       3.368874           3.413742          8
7          SeasonalNaive   -1.024961  -1.044558        0.026598       0.025762           0.083555          1
8       RecursiveTa

# Prediction

In [70]:
# Prepare forecast horizon covariates
future_index = get_forecast_horizon_index_ts_dataframe(sales_data, prediction_length=28)
future_timestamps = future_index.get_level_values("timestamp")

# Create a DataFrame for future timestamps to merge with calendar
future_dates_df = pd.DataFrame({'timestamp': future_timestamps})

# Ensure 'date' in calendar is in datetime format and set as index
calendar['date'] = pd.to_datetime(calendar.index)  # Ensure date is datetime if not already
calendar.set_index('date', inplace=True)

# Select relevant columns from calendar for merging
calendar_filtered = calendar[['wday', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'] + event_columns]

# Merge future_dates_df with calendar_filtered on 'timestamp'
known_covariates = future_dates_df.merge(calendar_filtered, left_on='timestamp', right_index=True, how='left')

# Set the index for known_covariates to match future_index
known_covariates.index = future_index


In [71]:
known_covariates

timestamp  wday  snap_CA  snap_TX  \
item_id                     timestamp                                       
FOODS_3_586_TX_1_evaluation 2016-05-23 2016-05-23     3        0        0   
                            2016-05-24 2016-05-24     4        0        0   
                            2016-05-25 2016-05-25     5        0        0   
                            2016-05-26 2016-05-26     6        0        0   
                            2016-05-27 2016-05-27     7        0        0   
                            2016-05-28 2016-05-28     1        0        0   
                            2016-05-29 2016-05-29     2        0        0   
                            2016-05-30 2016-05-30     3        0        0   
                            2016-05-31 2016-05-31     4        0        0   
                            2016-06-01 2016-06-01     5        1        1   
                            2016-06-02 2016-06-02     6        1        0   
                            2016-06-03 2016-06-03     7        1        1   
                            2016-06-04 2016-06-04     1        1        0   
                            2016-06-05 2016-06-05     2        1        1   
                            2016-06-06 2016-06-06     3        1        1   
                            2016-06-07 2016-06-07     4        1        1   
                            2016-06-08 2016-06-08     5        1        0   
                            2016-06-09 2016-06-09     6        1        1   
                            2016-06-10 2016-06-10     7        1        0   
                            2016-06-11 2016-06-11     1        0        1   
                            2016-06-12 2016-06-12     2        0        1   
                            2016-06-13 2016-06-13     3        0        1   
                            2016-06-14 2016-06-14     4        0        0   
                            2016-06-15 2016-06-15     5        0        1   
                            2016-06-16 2016-06-16     6        0        0   
                            2016-06-17 2016-06-17     7        0        0   
                            2016-06-18 2016-06-18     1        0        0   
                            2016-06-19 2016-06-19     2        0        0   

                                        snap_WI  sell_price  event_name_1  \
item_id                     timestamp                                       
FOODS_3_586_TX_1_evaluation 2016-05-23        0    1.679688            30   
                            2016-05-24        0    1.679688            30   
                            2016-05-25        0    1.679688            30   
                            2016-05-26        0    1.679688            30   
                            2016-05-27        0    1.679688            30   
                            2016-05-28        0    1.679688            30   
                            2016-05-29        0    1.679688            30   
                            2016-05-30        0    1.679688            14   
                            2016-05-31        0    1.679688            30   
                            2016-06-01        0    1.679688            30   
                            2016-06-02        1    1.679688            17   
                            2016-06-03        1    1.679688            30   
                            2016-06-04        0    1.679688            30   
                            2016-06-05        1    1.679688            30   
                            2016-06-06        1    1.679688            30   
                            2016-06-07        0    1.679688            24   
                            2016-06-08        1    1.679688            30   
                            2016-06-09        1    1.679688            30   
                            2016-06-10        0    1.679688            30   
                            2016-06-11        1    1.679688            30   
                            2016-06

In [72]:
# Generate predictions
predictions = predictor.predict(sales_data, known_covariates=known_covariates)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [73]:
import plotly.graph_objects as go

# Reset index to make 'timestamp' a simple column for plotting
sales_data_reset = sales_data.reset_index()

# Create traces for actual and predicted sales
fig = go.Figure()

# Add actual sales trace
fig.add_trace(go.Scatter(
    x=sales_data_reset['timestamp'][-28:], 
    y=sales_data_reset['sales'][-28:], 
    mode='lines+markers',
    name='Actual Sales'
))

# Add predicted sales trace
fig.add_trace(go.Scatter(
    x=sales_data_reset['timestamp'][-28:], 
    y=predictions['mean'], 
    mode='lines+markers',
    name='Predicted Sales'
))

# Update layout with titles and axis labels
fig.update_layout(
    title="Actual vs Predicted Sales with Known Covariates (AutoGluon)",
    xaxis_title="Date",
    yaxis_title="Sales",
    legend_title="Legend",
    template="plotly_dark"
)

# Show plot
fig.show()


# Applying on all validation set

In [2]:
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
sales_train_evaluation = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
sell_prices = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")

In [3]:
# Memory optimization function
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min, c_max = df[col].min(), df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
        elif col_type == 'object':
            if col == 'date':
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            else:
                df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Memory usage reduced to {end_mem:5.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df

In [4]:
# Apply the optimized memory reduction function to each dataframe
calendar = reduce_mem_usage(calendar)
sell_prices = reduce_mem_usage(sell_prices)
sales_train_evaluation = reduce_mem_usage(sales_train_evaluation)

Memory usage reduced to  0.13 Mb (40.4% reduction)
Memory usage reduced to 45.76 Mb (78.1% reduction)
Memory usage reduced to 96.30 Mb (78.7% reduction)


In [5]:
sales_train_evaluation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [6]:
# Step 1: Map d_ columns to dates
date_mapping = calendar[['date', 'd']].set_index('d').to_dict()['date']

# Step 2: Extract only the sales data columns and transpose
sales_data = sales_train_evaluation.filter(regex=r'd_\d+', axis=1).T
sales_data.index.name = 'd'  # Name the index for clarity

# Step 3: Map the index (d_1, d_2, ...) to actual dates using date_mapping
sales_data['timestamp'] = sales_data.index.map(date_mapping)
sales_data['timestamp'] = pd.to_datetime(sales_data['timestamp'])

# **Filter to keep only data from 2013-01-01 onwards**
sales_data = sales_data[sales_data['timestamp'] >= '2013-01-01']

# Step 4: Reset the index and set item_ids as columns
item_ids = sales_train_evaluation['id'].values
sales_data.columns = list(item_ids) + ['timestamp']  # Append 'timestamp' to match the shape

# Step 5: Set timestamp as the index
sales_data = sales_data.set_index('timestamp')

# Step 6: Convert to long format
sales_data_long = sales_data.reset_index().melt(id_vars='timestamp', var_name='item_id', value_name='sales')

# Step 7: Convert to TimeSeriesDataFrame for use with AutoGluon
sales_data = TimeSeriesDataFrame(sales_data_long.set_index(['item_id', 'timestamp']))


In [7]:
sales_data = reduce_mem_usage(sales_data)

Memory usage reduced to 217.27 Mb (0.0% reduction)


In [8]:
sales_data

sales
item_id                       timestamp        
HOBBIES_1_001_CA_1_evaluation 2013-01-01      0
                              2013-01-02      0
                              2013-01-03      0
                              2013-01-04      0
                              2013-01-05      0
...                                         ...
FOODS_3_827_WI_3_evaluation   2016-05-18      0
                              2016-05-19      2
                              2016-05-20      2
                              2016-05-21      5
                              2016-05-22      1

[37746620 rows x 1 columns]

# Setting up time-varying covariates

In [9]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [10]:
sell_prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [11]:
# Step 1: Encode event-related features in the calendar dataframe
event_columns = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
le = LabelEncoder()
for col in event_columns:
    calendar[col] = le.fit_transform(calendar[col].astype(str))

# Step 2: Merge calendar information into sell_prices on 'wm_yr_wk'
# This avoids expanding rows as 'sell_prices' has the required granularity
sell_prices = sell_prices.merge(calendar, how='left', on='wm_yr_wk')

In [12]:
sell_prices

,store_id,item_id,wm_yr_wk,sell_price,date,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,CA_1,HOBBIES_1_001,11325,9.578125,2013-07-13,Saturday,1,7,2013,d_897,30,4,4,2,0,1,0
1,CA_1,HOBBIES_1_001,11325,9.578125,2013-07-14,Sunday,2,7,2013,d_898,30,4,4,2,0,0,1
2,CA_1,HOBBIES_1_001,11325,9.578125,2013-07-15,Monday,3,7,2013,d_899,30,4,4,2,0,1,1
3,CA_1,HOBBIES_1_001,11325,9.578125,2013-07-16,Tuesday,4,7,2013,d_900,30,4,4,2,0,0,0
4,CA_1,HOBBIES_1_001,11325,9.578125,2013-07-17,Wednesday,5,7,2013,d_901,30,4,4,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,WI_3,FOODS_3_827,11620,1.000000,2016-06-15,Wednesday,5,6,2016,d_1965,30,4,4,2,0,1,1
47735393,WI_3,FOODS_3_827,11620,1.000000,2016-06-16,Thursday,6,6,2016,d_1966,30,4,4,2,0,0,0
47735394,WI_3,FOODS_3_827,11620,1.000000,2016-06-17,Friday,7,6,2016,d_1967,30,4,4,2,0,0,0
47735395,WI_3,FOODS_3_827,11621,1.000000,2016-06-18,Saturday,1,6,2016,d_1968,30,4,4,2,0,0,0


# Keep only data from 2013-01-01

In [13]:
# Convert 'item_id' and 'store_id' to strings before concatenation
sell_prices['item_id'] = sell_prices['item_id'].astype(str)
sell_prices['store_id'] = sell_prices['store_id'].astype(str)

# Create the new column with the required format
sell_prices['item_id_full'] = sell_prices['item_id'] + "_" + sell_prices['store_id'] + "_evaluation"

# Convert the 'date' column in sell_prices to datetime if not already done
sell_prices['date'] = pd.to_datetime(sell_prices['date'])

# Filter sell_prices to keep only data from 2013-01-01 onwards
sell_prices = sell_prices[sell_prices['date'] >= '2013-01-01']

# Now filter only relevant columns and rename 'date' to 'timestamp' for merging
calendar_columns = ['wday', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
sell_prices_filtered = sell_prices[['item_id_full', 'date'] + calendar_columns].rename(columns={'date': 'timestamp'})

# Reset index for sales_data for merging
sales_data_reset = sales_data.reset_index()

In [14]:
sales_data_reset = reduce_mem_usage(sales_data_reset)

Memory usage reduced to 433.22 Mb (33.1% reduction)


In [15]:
sell_prices_filtered = reduce_mem_usage(sell_prices_filtered)

Memory usage reduced to 946.57 Mb (54.8% reduction)


In [16]:
sales_data_reset

,item_id,timestamp,sales
0,HOBBIES_1_001_CA_1_evaluation,2013-01-01,0
1,HOBBIES_1_001_CA_1_evaluation,2013-01-02,0
2,HOBBIES_1_001_CA_1_evaluation,2013-01-03,0
3,HOBBIES_1_001_CA_1_evaluation,2013-01-04,0
4,HOBBIES_1_001_CA_1_evaluation,2013-01-05,0
...,...,...,...
37746615,FOODS_3_827_WI_3_evaluation,2016-05-18,0
37746616,FOODS_3_827_WI_3_evaluation,2016-05-19,2
37746617,FOODS_3_827_WI_3_evaluation,2016-05-20,2
37746618,FOODS_3_827_WI_3_evaluation,2016-05-21,5


In [17]:
sell_prices_filtered

,item_id_full,timestamp,wday,snap_CA,snap_TX,snap_WI,sell_price,event_name_1,event_type_1,event_name_2,event_type_2
0,HOBBIES_1_001_CA_1_evaluation,2013-07-13,1,0,1,0,9.578125,30,4,4,2
1,HOBBIES_1_001_CA_1_evaluation,2013-07-14,2,0,0,1,9.578125,30,4,4,2
2,HOBBIES_1_001_CA_1_evaluation,2013-07-15,3,0,1,1,9.578125,30,4,4,2
3,HOBBIES_1_001_CA_1_evaluation,2013-07-16,4,0,0,0,9.578125,30,4,4,2
4,HOBBIES_1_001_CA_1_evaluation,2013-07-17,5,0,0,0,9.578125,30,4,4,2
...,...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_827_WI_3_evaluation,2016-06-15,5,0,1,1,1.000000,30,4,4,2
47735393,FOODS_3_827_WI_3_evaluation,2016-06-16,6,0,0,0,1.000000,30,4,4,2
47735394,FOODS_3_827_WI_3_evaluation,2016-06-17,7,0,0,0,1.000000,30,4,4,2
47735395,FOODS_3_827_WI_3_evaluation,2016-06-18,1,0,0,0,1.000000,30,4,4,2


In [18]:
del sell_prices
gc.collect()

0

# Merge by batch and monitor memory

In [19]:
# Define chunk size (adjust based on available memory)
chunk_size = 1_000_000

# Initialize an empty list to store merged chunks
merged_chunks = []

# Total rows in sales_data_reset for progress tracking
total_rows = len(sales_data_reset)
num_chunks = (total_rows // chunk_size) + 1

# Loop over sales_data_reset in chunks
for i, start in enumerate(range(0, total_rows, chunk_size)):
    # Slice the chunk from sales_data_reset
    sales_data_chunk = sales_data_reset.iloc[start:start + chunk_size]
    
    # Print progress
    print(f"Processing chunk {i + 1}/{num_chunks} (rows {start} to {start + chunk_size - 1})")

    # Perform the merge for the chunk with sell_prices_filtered
    merged_chunk = sales_data_chunk.merge(
        sell_prices_filtered,
        left_on=['item_id', 'timestamp'],
        right_on=['item_id_full', 'timestamp'],
        how='left'
    )

    # Drop unnecessary columns and append the chunk to the list
    merged_chunk.drop(columns=['item_id_full'], inplace=True)
    merged_chunks.append(merged_chunk)

    # Print memory usage after appending the chunk
    mem = psutil.virtual_memory()
    print(f"Chunk {i + 1} merged and added to list. Current memory usage: {mem.percent}%")

    # Delete the intermediate variables and run garbage collection
    del sales_data_chunk, merged_chunk
    gc.collect()

# Concatenate all merged chunks
print("Concatenating all chunks into final DataFrame...")
sales_data = pd.concat(merged_chunks).set_index(['item_id', 'timestamp'])

# Optionally, clear the merged_chunks list to free up memory
del merged_chunks
gc.collect()

# Print final memory usage
mem = psutil.virtual_memory()
print(f"All chunks processed and merged. Final memory usage: {mem.percent}%")

Processing chunk 1/38 (rows 0 to 999999)
Chunk 1 merged and added to list. Current memory usage: 54.6%
Processing chunk 2/38 (rows 1000000 to 1999999)
Chunk 2 merged and added to list. Current memory usage: 54.8%
Processing chunk 3/38 (rows 2000000 to 2999999)
Chunk 3 merged and added to list. Current memory usage: 54.9%
Processing chunk 4/38 (rows 3000000 to 3999999)
Chunk 4 merged and added to list. Current memory usage: 55.2%
Processing chunk 5/38 (rows 4000000 to 4999999)
Chunk 5 merged and added to list. Current memory usage: 55.3%
Processing chunk 6/38 (rows 5000000 to 5999999)
Chunk 6 merged and added to list. Current memory usage: 55.5%
Processing chunk 7/38 (rows 6000000 to 6999999)
Chunk 7 merged and added to list. Current memory usage: 55.7%
Processing chunk 8/38 (rows 7000000 to 7999999)
Chunk 8 merged and added to list. Current memory usage: 56.0%
Processing chunk 9/38 (rows 8000000 to 8999999)
Chunk 9 merged and added to list. Current memory usage: 56.2%
Processing chunk 

In [21]:
sales_data

sales  wday  snap_CA  snap_TX  \
item_id                       timestamp                                   
HOBBIES_1_001_CA_1_evaluation 2013-01-01      0   NaN      NaN      NaN   
                              2013-01-02      0   NaN      NaN      NaN   
                              2013-01-03      0   NaN      NaN      NaN   
                              2013-01-04      0   NaN      NaN      NaN   
                              2013-01-05      0   NaN      NaN      NaN   
...                                         ...   ...      ...      ...   
FOODS_3_827_WI_3_evaluation   2016-05-18      0   5.0      0.0      0.0   
                              2016-05-19      2   6.0      0.0      0.0   
                              2016-05-20      2   7.0      0.0      0.0   
                              2016-05-21      5   1.0      0.0      0.0   
                              2016-05-22      1   2.0      0.0      0.0   

                                          snap_WI  sell_price  event_name_1  \
item_id                       timestamp                                       
HOBBIES_1_001_CA_1_evaluation 2013-01-01      NaN         NaN           NaN   
                              2013-01-02      NaN         NaN           NaN   
                              2013-01-03      NaN         NaN           NaN   
                              2013-01-04      NaN         NaN           NaN   
                              2013-01-05      NaN         NaN           NaN   
...                                           ...         ...           ...   
FOODS_3_827_WI_3_evaluation   2016-05-18      0.0         1.0          30.0   
                              2016-05-19      0.0         1.0          30.0   
                              2016-05-20      0.0         1.0          30.0   
                              2016-05-21      0.0         1.0          30.0   
                              2016-05-22      0.0         1.0          30.0   

                                          event_type_1  event_name_2  \
item_id                       timestamp                                
HOBBIES_1_001_CA_1_evaluation 2013-01-01           NaN           NaN   
                              2013-01-02           NaN           NaN   
                              2013-01-03           NaN           NaN   
                              2013-01-04           NaN           NaN   
                              2013-01-05           NaN           NaN   
...                                                ...           ...   
FOODS_3_827_WI_3_evaluation   2016-05-18           4.0           4.0   
                              2016-05-19           4.0           4.0   
                              2016-05-20           4.0           4.0   
                              2016-05-21           4.0           4.0   
                              2016-05-22           4.0           4.0   

                                          event_type_2  
item_id                       timestamp                 
HOBBIES_1_001_CA_1_evaluation 2013-01-01           NaN  
                              2013-01-02           NaN  
                              2013-01-03           NaN  
                              2013-01-04           NaN  
                              2013-01-05           NaN  
...                                                ...  
FOODS_3_827_WI_3_evaluation   2016-05-18           2.0  
                              2016-05-19           2.0  
                              2016-05-20           2.0  
                              2016-05-21           2.0  
                              2016-05-22           2.0  

[37746620 rows x 10 columns]

In [22]:
del sales_data_reset, sell_prices_filtered
gc.collect()

0

# Initialise and fit with TimeSeriesPredictor

In [ ]:
# Step 1: Initialize the TimeSeriesPredictor with a primary eval_metric
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="sales",
    known_covariates_names=['wday', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'] + event_columns,
    eval_metric="RMSSE"  # Primary metric for training
)

# Step 2: Fit the predictor
predictor.fit(train_data=sales_data, presets="medium_quality", time_limit=600)

No path specified. Models will be saved in: "AutogluonModels/ag-20241105_085720"
Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'AutogluonModels/ag-20241105_085720'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
GPU Count:          0
Memory Avail:       8.88 GB / 31.36 GB (28.3%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSSE,
 'hyperparameters': 'light',
 'known_covariates_names': ['wday',
                            'snap_CA',
                            'snap_TX',
                            'snap_WI',
                            'sell_price',
                            'event_name_1',
                            'event_type_1',
                           